In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB

In [2]:
#nombre de columnas
colnames = ['deliverer_id', 'delivery_zone', 'monthly_app_usage', 'suscription_type', 'paid_price', 'customer_size', 'menu', 'delay_time']

#leemos dataframe y asignamos nombre a las columnas
df_train_delivery = pd.read_csv('train_delivery_data.csv', header=None, names=colnames)

df_train_delivery.head()

,deliverer_id,delivery_zone,monthly_app_usage,suscription_type,paid_price,customer_size,menu,delay_time
0,0,VI,17,Semestral,16.940991,4,French,11.332126
1,98,VII,18,Free,39.064177,5,Asian,10.429260
2,14,IV,16,Trimestral,32.826544,2,Italian,7.900892
3,72,V,17,Free,7.556762,3,Mexican,14.689806
4,11,VIII,12,Semestral,34.200684,1,Indian,12.513841


In [3]:
#recodificación vector obetivo
# 1 para demora > promedio
# 0 para demora =< promedio
df_train_delivery.delay_time = np.where(df_train_delivery.delay_time > df_train_delivery.delay_time.mean(), 1, 0)

#df_train_delivery.head()

In [4]:
#buscamos las variables que sean de tipo string 
dict_data_type = dict(df_train_delivery.dtypes)
str_var = []

for colname in dict_data_type:
    if dict_data_type[colname] == 'O':
        str_var.append(colname)

#Recodificamos los atributos de tipo string en K-1 columnas 
df_train_delivery = pd.get_dummies(df_train_delivery, columns= str_var, drop_first=True)

#df_train_delivery.head()

In [5]:
#Definimos vector Objetivo
vector_objetivo = 'delay_time'

#Enlistamos y removemos vector objetivo de la matriz de entrenamiento
colname_x = list(df_train_delivery.columns)
colname_x.remove(vector_objetivo)

#definimos matriz de entrenamiento y vector objetivo
x_matriz = df_train_delivery[colname_x]
y_vector = df_train_delivery[vector_objetivo]

#Generamos conjunto de entrenamiento y validación 
x_train, x_test, y_train, y_test = train_test_split(x_matriz, y_vector, test_size=0.33, random_state=11238)

In [7]:
#Entrenamiento de modelos
model_logit = LogisticRegression().fit(x_train, y_train)
model_decisiontree = DecisionTreeClassifier().fit(x_train, y_train)
model_randomforest = RandomForestClassifier().fit(x_train, y_train)
model_gradientboost = GradientBoostingClassifier().fit(x_train, y_train)
model_bernoulli = BernoulliNB().fit(x_train, y_train)

models = [model_logit, model_decisiontree, model_randomforest, model_gradientboost, model_bernoulli]

for model in models:
    print(f'Classification report: {type(model).__name__}\r\n\n{classification_report(y_test, model.predict(x_test))}\n\n\n')
    

C:\Users\const\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification report: LogisticRegression

              precision    recall  f1-score   support

           0       0.46      0.56      0.51       167
           1       0.42      0.32      0.36       163

    accuracy                           0.44       330
   macro avg       0.44      0.44      0.43       330
weighted avg       0.44      0.44      0.43       330




Classification report: DecisionTreeClassifier

              precision    recall  f1-score   support

           0       0.54      0.44      0.49       167
           1       0.52      0.62      0.57       163

    accuracy                           0.53       330
   macro avg       0.53      0.53      0.53       330
weighted avg       0.53      0.53      0.53       330




Classification report: RandomForestClassifier

              precision    recall  f1-score   support

           0       0.53      0.53      0.53       167
           1       0.52      0.52      0.52       163

    accuracy                           

In [9]:
pickle.dump(model_bernoulli, open("Bernoulli.pkl" , 'wb'))